Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [1]:
import os
import sys
nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.squad import load_pandas_df
from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.models.bert.question_answering import BERTQAExtractor
from utils_nlp.models.bert.qa_utils import postprocess_answers, evaluate_qa

In [2]:
SQUAD_VERSION = "v1.1" 
CACHE_DIR = "./temp"

LANGUAGE = Language.ENGLISHLARGEWWM
DO_LOWER_CASE = True

MAX_SEQ_LENGTH = 384
NUM_EPOCHS = 2
BATCH_SIZE = 16
LEARNING_RATE = 3e-5
WARMUP = 0.1

DOC_TEXT_COL = "doc_text"
QUESTION_TEXT_COL = "question_text"
ANSWER_START_COL = "answer_start"
ANSWER_TEXT_COL = "answer_text"
QA_ID_COL = "qa_id"
IS_IMPOSSIBLE_COL = "is_impossible"

In [3]:
train_df = load_pandas_df(local_cache_path=".", squad_version="v1.1", file_split="train")
dev_df = load_pandas_df(local_cache_path=".", squad_version="v1.1", file_split="dev")

In [4]:
train_df.head()

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,5733be284776f41900661182,False
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,5733be284776f4190066117f,False
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,5733be284776f41900661180,False
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,5733be284776f41900661181,False
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,5733be284776f4190066117e,False


In [5]:
dev_df.head()

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"[177, 177, 177]","[Denver Broncos, Denver Broncos, Denver Broncos]",56be4db0acb8001400a502ec,False
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"[249, 249, 249]","[Carolina Panthers, Carolina Panthers, Carolin...",56be4db0acb8001400a502ed,False
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"[403, 355, 355]","[Santa Clara, California, Levi's Stadium, Levi...",56be4db0acb8001400a502ee,False
3,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"[177, 177, 177]","[Denver Broncos, Denver Broncos, Denver Broncos]",56be4db0acb8001400a502ef,False
4,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"[488, 488, 521]","[gold, gold, gold]",56be4db0acb8001400a502f0,False


In [6]:
tokenizer = Tokenizer(language=LANGUAGE, to_lower=DO_LOWER_CASE, cache_dir=CACHE_DIR)

In [7]:
train_features, qa_examples = tokenizer.tokenize_qa(
    doc_text=train_df[DOC_TEXT_COL], 
    question_text=train_df[QUESTION_TEXT_COL], 
    answer_start=train_df[ANSWER_START_COL], 
    answer_text=train_df[ANSWER_TEXT_COL],
    qa_id=train_df[QA_ID_COL],
    is_impossible=train_df[IS_IMPOSSIBLE_COL],
    is_training=True,
    max_len=MAX_SEQ_LENGTH)

In [8]:
dev_features, dev_examples = tokenizer.tokenize_qa(
    doc_text=dev_df[DOC_TEXT_COL], 
    question_text=dev_df[QUESTION_TEXT_COL], 
    answer_start=dev_df[ANSWER_START_COL], 
    answer_text=dev_df[ANSWER_TEXT_COL],
    qa_id=dev_df[QA_ID_COL],
    is_impossible=dev_df[IS_IMPOSSIBLE_COL],
    is_training=False,
    max_len=MAX_SEQ_LENGTH)

In [9]:
sample_feature = dev_features[0]
for f in type(sample_feature)._fields:
    print(f)
    print(getattr(sample_feature, f))
    print()

unique_id
1000000000

example_index
0

tokens
['[CLS]', 'which', 'nfl', 'team', 'represented', 'the', 'afc', 'at', 'super', 'bowl', '50', '?', '[SEP]', 'super', 'bowl', '50', 'was', 'an', 'american', 'football', 'game', 'to', 'determine', 'the', 'champion', 'of', 'the', 'national', 'football', 'league', '(', 'nfl', ')', 'for', 'the', '2015', 'season', '.', 'the', 'american', 'football', 'conference', '(', 'afc', ')', 'champion', 'denver', 'broncos', 'defeated', 'the', 'national', 'football', 'conference', '(', 'nfc', ')', 'champion', 'carolina', 'panthers', '24', '–', '10', 'to', 'earn', 'their', 'third', 'super', 'bowl', 'title', '.', 'the', 'game', 'was', 'played', 'on', 'february', '7', ',', '2016', ',', 'at', 'levi', "'", 's', 'stadium', 'in', 'the', 'san', 'francisco', 'bay', 'area', 'at', 'santa', 'clara', ',', 'california', '.', 'as', 'this', 'was', 'the', '50th', 'super', 'bowl', ',', 'the', 'league', 'emphasized', 'the', '"', 'golden', 'anniversary', '"', 'with', 'various', 'g

In [10]:
# train_features = train_features[0:63]

In [11]:
qa_extractor = BERTQAExtractor(language=LANGUAGE, cache_dir=CACHE_DIR)

In [ ]:
qa_extractor.fit(train_features,
                 num_epochs=NUM_EPOCHS,
                 batch_size=BATCH_SIZE,
                 lr=LEARNING_RATE,
                 warmup_proportion=WARMUP,
                 model_output_dir=CACHE_DIR)

Iteration:   0%|          | 0/5541 [00:00<?, ?it/s]/data/anaconda/envs/nlp_gpu/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

Iteration:   2%|▏         | 118/5541 [02:20<1:35:03,  1.05s/it]


Iteration:   4%|▍         | 240/5541 [04:36<1:33:15,  1.06s/it]


Iteration:   7%|▋         | 362/5541 [06:50<1:30:30,  1.05s/it]


Iteration:   9%|▊         | 484/5541 [09:06<1:28:28,  1.05s/it]


Iteration:  11%|█         | 606/5541 [11:23<1:26:39,  1.05s/it]


...

Iteration:  35%|███▍      | 1932/5541 [35:55<1:03:18,  1.05s/it]


Iteration:  37%|███▋      | 2052/5541 [38:09<1:01:43,  1.06s/it]


Iteration:  39%|███▉      | 2173/5541 [40:20<58:53,  1.05s/it]


Iteration:  41%|████▏     | 2296/5541 [42:39<1:03:12,  1.17s/it]


Iteration:  43%|████▎     | 2394/5541 [44:26<55:35,  1.06s/it]

In [ ]:
qa_results = qa_extractor.predict(dev_features)

In [ ]:
final_answers = postprocess_answers(dev_examples, 
                                    dev_features, 
                                    qa_results, 
                                    do_lower_case=DO_LOWER_CASE)

In [ ]:
# final_answers

In [ ]:
# import json
# with open('/home/hlu/models/wwm_uncased_fintuned_squad/predictions_.json') as f:
#     preds = json.load(f)

In [ ]:
evaluate_qa(qa_ids=dev_df['qa_id'], 
            actuals=dev_df['answer_text'], 
            preds=final_answers)